<a href="https://colab.research.google.com/github/peremartra/FinLLMOpt/blob/main/Llama-FinSent-S/03_Example_Usages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLama-FinSent-S
## Example usage.

In [1]:
!pip install -q transformers==4.47.1
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
#Load Model from Hugging Face.
model_name = "oopere/Llama-FinSent-S"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=4916, bias=False)
          (up_proj): Linear(in_features=2048, out_features=4916, bias=False)
          (down_proj): Linear(in_features=4916, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [66]:
newline_token_id = tokenizer.encode("\n", add_special_tokens=False)[0]
def generate_response(prompt, model):
    # Format prompt using chat template
    #eot_id = tokenizer.convert_tokens_to_ids(["<|eot_id|>"])[0]

    full_prompt="""Instruction: What is the sentiment of this news?
    Please choose an answer from {negative/neutral/positive}.""" + "\n" + "News: " + prompt + "\n" + "Answer:"
    print(full_prompt)

    inputs = tokenizer(full_prompt, return_tensors="pt").to(device)

    # Generate with strict stopping criteria
    outputs = model.generate(
        **inputs,
        max_new_tokens=15,
        eos_token_id=newline_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        num_beams=1,
        temperature=0.001,  # More deterministic
        no_repeat_ngram_size=3,
        early_stopping=True,
    )

    # Decode and extract answer
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=False)

    return full_response.split("Answer:")[-1].split("\n")[0].strip(), full_response

In [67]:
# Example usage
test_news = "Ahlstrom Corporation STOCK EXCHANGE ANNOUNCEMENT 7.2.2007 at 10.30 A total of 56,955 new shares of A..."
response, full = generate_response(test_news, model)
print(response)

Instruction: What is the sentiment of this news? 
    Please choose an answer from {negative/neutral/positive}.
News: Ahlstrom Corporation STOCK EXCHANGE ANNOUNCEMENT 7.2.2007 at 10.30 A total of 56,955 new shares of A...
Answer:
neutral


In [68]:
test_news = "$ESI on lows, down $1.50 to $2.50 BK a real's possibility"
response, full = generate_response(test_news, model)
print(response)

Instruction: What is the sentiment of this news? 
    Please choose an answer from {negative/neutral/positive}.
News: $ESI on lows, down $1.50 to $2.50 BK a real's possibility
Answer:
negative


In [69]:
test_news ="""
The world 's second largest stainless steel maker said net profit in the three-month period until Dec.
31 surged to euro603 million US$ 781 million , or euro3 .33 US$ 4.31 per share ,
from euro172 million , or euro0 .94 per share , the previous year .
"""
response, full = generate_response(test_news, model)
print(response)

Instruction: What is the sentiment of this news? 
    Please choose an answer from {negative/neutral/positive}.
News: 
The world 's second largest stainless steel maker said net profit in the three-month period until Dec. 
31 surged to euro603 million US$ 781 million , or euro3 .33 US$ 4.31 per share , 
from euro172 million , or euro0 .94 per share , the previous year . 

Answer:
positive
